#Importing Libraries

In [ ]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import time
from tqdm import tqdm

#Loading the images

In [ ]:
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((224, 224))
    image = np.array(image)
    image = image / 255.0  # Scale to [0, 1]
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = tf.expand_dims(image, axis=0)  # Add batch dimension
    return image

#Visualization

In [ ]:
def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)
    plt.imshow(image)
    if title:
        plt.title(title)

content_path = "download.png"
style_path = "picasso.jpg"
content_image = load_image(content_path)
style_image = load_image(style_path)
print(content_image.shape, style_image.shape)

plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
imshow(content_image, "Content Image")
plt.subplot(1, 2, 2)
imshow(style_image, "Style Image")
plt.show()

#Feature Extraction


In [ ]:
vgg = tf.keras.applications.VGG19(weights="imagenet", include_top=False)
vgg.trainable = False

content_layers = ["block5_conv2"]
style_layers = ["block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

def vgg_layers(layer_names):
    vgg = tf.keras.applications.VGG19(include_top=False, weights="imagenet")
    vgg.trainable = False
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image * 255)

def gram_matrix(input_tensor):
    result = tf.linalg.einsum("bijc,bijd->bcd", input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

#Model

In [ ]:
class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        inputs = inputs * 255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers], outputs[self.num_style_layers:])
        style_outputs = [gram_matrix(style_output) for style_output in style_outputs]
        content_dict = {content_name: value for content_name, value in zip(self.content_layers, content_outputs)}
        style_dict = {style_name: value for style_name, value in zip(self.style_layers, style_outputs)}
        return {"content": content_dict, "style": style_dict}

extractor = StyleContentModel(style_layers, content_layers)
style_targets = extractor(style_image)["style"]
content_targets = extractor(content_image)["content"]

def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

optim = tf.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

#Loss

In [ ]:
style_weight = 1
content_weight = 1e2

def style_content_loss(outputs):
    style_outputs = outputs["style"]
    content_outputs = outputs["content"]
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name] - style_targets[name])**2) for name in style_outputs.keys()])
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name] - content_targets[name])**2) for name in content_outputs.keys()])
    style_loss *= style_weight / num_style_layers
    content_loss *= content_weight / num_content_layers
    return style_loss + content_loss

#Training

In [ ]:
@tf.function
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs)
        loss += 1e-6 * tf.image.total_variation(image)
    grad = tape.gradient(loss, image)
    optim.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return Image.fromarray(tensor)
image = tf.Variable(content_image)
import IPython.display as display
from tqdm import tqdm  # For progress bar
epochs = 10
steps_per_epoch = 150

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step in tqdm(range(steps_per_epoch), desc="Training Steps"):
        train_step(image)
        if step % 10 == 0:  # Display progress every 10 steps
            display.clear_output(wait=True)
            display.display(tensor_to_image(image))

    print(f"Done with epoch {epoch + 1}")

#Saving the Image

In [ ]:
display.display(tensor_to_image(image))
final_image = tensor_to_image(image)
final_image.save("stylized_image.jpg")

##Improving the image

In [ ]:
from PIL import Image, ImageEnhance

# Enhance sharpness
enhancer = ImageEnhance.Sharpness(image)
sharpness_factor = 2.0  # Increase to enhance sharpness, decrease to reduce sharpness
image = enhancer.enhance(sharpness_factor)

# Enhance contrast
enhancer = ImageEnhance.Contrast(image)
contrast_factor = 1.5  # Increase to enhance contrast, decrease to reduce contrast
image = enhancer.enhance(contrast_factor)

# Save the improved image
output_path = "improved_image.jpg"
image.save(output_path)

# Display the improved image
image.show()
